## Deliverable 2. Create a Customer Travel Destinations Map.

In [32]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,74.70,82,100,18.07,overcast clouds
1,1,Nabire,ID,-3.3667,135.4833,74.98,91,100,4.07,overcast clouds
2,2,Mildura,AU,-34.2000,142.1500,51.76,62,4,3.44,clear sky
3,3,Ouahigouya,BF,13.5828,-2.4216,74.95,14,46,10.42,scattered clouds
4,4,Aklavik,CA,68.2191,-135.0107,-11.20,84,20,5.75,few clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 75
What is your desired maximum temperature for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
14,14,Amapa,BR,1.0000,-52.0000,86.67,56,88,3.56,overcast clouds
16,16,Vila Velha,BR,-20.3297,-40.2925,75.15,92,90,8.99,overcast clouds
18,18,Butaritari,KI,3.0707,172.7902,81.97,75,100,18.21,overcast clouds
22,22,Georgetown,MY,5.4112,100.3354,78.73,87,20,5.01,few clouds
24,24,Jequitinhonha,BR,-16.4339,-41.0033,77.22,94,100,1.57,overcast clouds
27,27,Hermanus,ZA,-34.4187,19.2345,76.89,72,100,11.30,overcast clouds
35,35,Dingle,PH,10.9995,122.6711,75.34,91,45,7.16,scattered clouds
36,36,Hithadhoo,MV,-0.6000,73.0833,82.24,72,67,3.18,broken clouds
38,38,Domoni,KM,-12.2569,44.5319,80.06,80,38,1.88,scattered clouds
48,48,Castro,BR,-24.7911,-50.0119,79.12,38,0,3.53,clear sky


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
14,14,Amapa,BR,1.0000,-52.0000,86.67,56,88,3.56,overcast clouds
16,16,Vila Velha,BR,-20.3297,-40.2925,75.15,92,90,8.99,overcast clouds
18,18,Butaritari,KI,3.0707,172.7902,81.97,75,100,18.21,overcast clouds
22,22,Georgetown,MY,5.4112,100.3354,78.73,87,20,5.01,few clouds
24,24,Jequitinhonha,BR,-16.4339,-41.0033,77.22,94,100,1.57,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
684,684,Bida,NG,9.0833,6.0167,81.00,24,6,2.01,clear sky
691,691,Consolacion Del Sur,CU,22.5047,-83.5136,88.66,44,9,9.04,clear sky
699,699,Jaicos,BR,-7.3592,-41.1378,87.39,46,100,8.43,overcast clouds
703,703,Pangani,TZ,-5.4333,38.9667,79.29,88,61,8.30,light rain


In [10]:
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
14,Amapa,BR,86.67,overcast clouds,1.0000,-52.0000,
16,Vila Velha,BR,75.15,overcast clouds,-20.3297,-40.2925,
18,Butaritari,KI,81.97,overcast clouds,3.0707,172.7902,
22,Georgetown,MY,78.73,few clouds,5.4112,100.3354,
24,Jequitinhonha,BR,77.22,overcast clouds,-16.4339,-41.0033,
27,Hermanus,ZA,76.89,overcast clouds,-34.4187,19.2345,
35,Dingle,PH,75.34,scattered clouds,10.9995,122.6711,
36,Hithadhoo,MV,82.24,broken clouds,-0.6000,73.0833,
38,Domoni,KM,80.06,scattered clouds,-12.2569,44.5319,
48,Castro,BR,79.12,clear sky,-24.7911,-50.0119,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Cannot find hotel...skipping.")  
        

Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.


In [33]:
# 7. Drop the rows where there is no Hotel Name.
no_hotel_name = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = no_hotel_name.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
16,Vila Velha,BR,75.15,overcast clouds,-20.3297,-40.2925,Quality Suites Vila Velha
18,Butaritari,KI,81.97,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
22,Georgetown,MY,78.73,few clouds,5.4112,100.3354,Cititel Penang
24,Jequitinhonha,BR,77.22,overcast clouds,-16.4339,-41.0033,Hotel Bela Vista
27,Hermanus,ZA,76.89,overcast clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
...,...,...,...,...,...,...,...
684,Bida,NG,81.00,clear sky,9.0833,6.0167,Royal Suite
691,Consolacion Del Sur,CU,88.66,clear sky,22.5047,-83.5136,Cuarto de alquiler
699,Jaicos,BR,87.39,overcast clouds,-7.3592,-41.1378,Hotel Jaicós
703,Pangani,TZ,79.29,light rain,-5.4333,38.9667,Safari Mashariki Lodge


In [37]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [38]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [39]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))